In [ ]:
import netCDF4 as nc
from netCDF4 import Dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets.embed import embed_minimal_html
import ipywidgets as widgets
import os
import scipy.linalg
os.environ["PROJ_LIB"] = "C:\\Utilities\\Python\\Anaconda\\Library\\share"; #fixr
from mpl_toolkits.basemap import Basemap

# Pulling File Data and reshaping into space-time matrix

In [ ]:
file = 'data/air.mon.meanv3.nc'
data = nc.Dataset(file)

In [ ]:
lats = data.variables['lat'][:]
lons = data.variables['lon'][:]
air = data.variables['air'][:]
levels = data.variables['level'][:]



In [ ]:
airdata = []
for i in range(air.shape[0]):
    annual_air = np.array(air[i].flatten()).tolist()
    airdata.append(annual_air)
airdata= np.array(airdata).T

# Preweighted Computations

In [ ]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    clim = np.nanmean(airdata, axis=1) #calculates climatology
    sdev = np.nanstd(airdata, axis = 1) #calculates standard deviation
clim = np.reshape(clim, (clim.size, 1))
sdev = np.reshape(sdev, (sdev.size,1))

In [ ]:
np.seterr(divide= 'ignore', invalid = 'ignore')
stnd_anom = (airdata- clim)/sdev #computing standard anomalies by subtracting mean and dividing by standard dev

# Weighted computations

In [ ]:
# Constants
p = 101325e-2 #standard pressure
r = 8.31432 #gas constant
m = 0.0289644 #molar mass of air
g = 9.8 #gravity m/s^2

## Air heights (approximated with hypsometric equation. Assume $P_0 = 1013.25$ mbar and $h_0 = 0$ 

$$
P_n = P_{n-1}e^{\frac{-gM(h_n-h_{n-1})}{RT}}
$$

In [ ]:
airheights = []
for i in range(0, 360*181):
    airheights.append(r*clim[i]/(-g*m)*np.log(levels[int(i/(181*360))]/p))
for i in range(181*360, clim.size):
    airheights.append(r*clim[i]/(-g*m)*np.log(levels[int(i/(181*360))]/levels[int(i/(181*360))-1]) +airheights[i-(181*360)])
airheights = np.array(airheights)
airheights = np.reshape(airheights, (airheights.size, 1))

## Thickness: $\Delta z_{ij}$

In [ ]:
thickness = np.empty([stnd_anom.shape[0], 1])
thickness[0:181*360] = airheights[0:181*360]
for i in range(181*360, air[0].size):
    thickness[i] = airheights[i] - airheights[i-181*360]

## Accounting for changes in latitude: $\sqrt{\cos{\phi}}$

In [ ]:
xxair, yyair = np.meshgrid(lons, lats) #creates meshgrid

#Creating array associating all points with their latitude
lats4air = np.array(yyair).flatten().tolist()
lats4weighing = []
for i in range(levels.size):
    lats4weighing += lats4air
lats4weighing = np.array(lats4weighing)
lats4weighing.shape

#square root of the cosine of the latitude
weightedA = np.sqrt(np.cos(lats4weighing*np.pi/180))
weightedA = np.reshape(weightedA, (weightedA.size, 1))
weightedA.shape

## Density of air: $\rho$

* calculated using the NASA's Earth Atmosphere Model with given pressure and temperature. https://www.grc.nasa.gov/www/k-12/airplane/atmosmet.html

In [ ]:
density = np.empty([weightedA.shape[0], 1])
for i in range(0,air[0].size):
    density[i] =  levels[int(i/(181*360))]/(2.869*clim[i])

## Volume weighted Anomalies

In [ ]:
vweightedanom = stnd_anom * weightedA *np.sqrt(thickness) * np.sqrt(density)

# EOF computations

In [ ]:
df = pd.DataFrame(data = vweightedanom)
dropna = df.dropna() #dropping NaN values out of matrix to do computations otherwise all values will be NaN
NanlessAnom = dropna.to_numpy()
NanlessAnom.shape 

In [ ]:
Sigma = np.matmul(NanlessAnom.T, NanlessAnom) #covariance matrix
eigenvalues, eigenvectors = scipy.linalg.eig(Sigma) #obtaining eigenvalues and eigenvectors
eigenvectors = eigenvectors.T

In [ ]:
index = np.argsort(eigenvalues)[::-1] #sorting eigenvalues from largest to smallest
eigvals = eigenvalues[index] #reordering eigenvalues
eigvecs = eigenvectors[index] #reordering eigenvectors

In [ ]:
EOFS = []

for j in range(0,vweightedanom.shape[1]):
        EOFS.append(np.matmul(vweightedanom, eigvecs[j])/np.linalg.norm(np.matmul(NanlessAnom, eigvecs[j])))
EOF1 = np.array(EOFS).T
#Gets Geometric EOFs

In [ ]:
np.seterr(divide='ignore', invalid='ignore')
PhysicalEOFs = EOF1/(weightedA * np.sqrt(thickness) * np.sqrt(density)) #Gets Physical EOFs by dividing by weight factors